# Deep-Learning enhanced Dynamic Mode Decomposition
In this notebook we're going to reproduce the DMD presented in [1].

## Model architecture
1. MLP: Encode the input
2. DMD: Apply the DMD algorithm
3. MLP: Decode the DMD output

The idea is to pair DMD with an autoencoder whose job is to find an optimal encoding to improve DMD accuracy (both in reconstruction and accuracy). More insights in [1].

## Implementation
First of all we import a few modules.

In [1]:
import time
import os
import logging
from functools import wraps
from copy import deepcopy

import numpy as np
import torch
import torch.optim as optim
from torch.nn.functional import mse_loss
from torch.utils.data import DataLoader
from torch.nn import Sequential, ReLU, Linear, Module

from pydmd import DMD

logger = logging.getLogger()
logger.setLevel(logging.INFO)

We prepare a special decorator `@timer` which decorates function for which we'd like to know the execution time. We're going to use this in the `DLDMD` class further below.

In [3]:
def timer(func):
    @wraps(func)
    def timed_func(*args, **kwargs):
        start = time.time_ns()
        val = func(*args, **kwargs)
        dt_ms = (time.time_ns() - start) / 1_000_000
        return dt_ms, val

    return timed_func

Finally we define the `DLDMD` class (obvisouly extending `torch.nn.Module`). The compulsory parameters are:
+ The encoder module;
+ A DMD instance from PyDMD
+ The decoder module;

In addition, the user may provide:
+ 4 training weights for the different outputs given by DMD (reconstruction, prediction, phase space, encoding accuracy);
+ An optimizer (by default we use `Adam`);
+ A `dict` of optimizer parameters;
+ The number of snapshots to be predicted (and therefore to train upon).

`DLDMD` provides methods for the forward pass (`forward`) which specializes in `train_step` and `eval_step`. It handles the computation of the loss function, and model-saving as well.

In [38]:
class DLDMD(Module):
    def __init__(
        self,
        encoder,
        dmd,
        decoder,
        encoding_weight,
        reconstruction_weight,
        prediction_weight,
        phase_space_weight,
        optimizer=optim.Adam,
        optimizer_kwargs={"lr": 1.0e-3, "weight_decay": 1.0e-9},
        n_prediction_snapshots=1,
        eval_on_cpu=True,
    ):
        super().__init__()

        self._encoder = encoder
        self._decoder = decoder
        
        self._optimizer = optimizer(self.parameters(), **optimizer_kwargs)
        logger.info(f"Optimizer: {self._optimizer}")

        self._encoding_weight = encoding_weight
        self._reconstruction_weight = reconstruction_weight
        self._prediction_weight = prediction_weight
        self._phase_space_weight = phase_space_weight

        self._eval_on_cpu = eval_on_cpu

        logger.info(f"DMD instance: {type(dmd)}")
        self._dmd = dmd
        # a copy of dmd to be used only for evaluation
        self._eval_dmd = deepcopy(dmd)

        self._n_prediction_snapshots = n_prediction_snapshots
        logger.info(
            f"DMD will predict {n_prediction_snapshots} snapshots during training."
        )

        logger.info("----- DLDMD children -----")
        logger.info(tuple(self.children()))

    def forward(self, input):
        if input.ndim == 2:
            input = input[None]

        logger.debug(f"Input shape: {input.shape}")
        encoded_input = self._encoder(input.swapaxes(-1, -2))
        logger.debug(f"Encoded input shape: {encoded_input.shape}")
        self._dmd.fit(encoded_input.swapaxes(-1, -2), batch=True)
        self._dmd.dmd_time["tend"] = (
            self._dmd.original_time["tend"] + self._n_prediction_snapshots
        )

        encoded_output = self._dmd.reconstructed_data.swapaxes(-1, -2)
        logger.debug(f"Encoded output shape: {encoded_output.shape}")

        if not torch.is_complex(input):
            old_dtype = encoded_output.dtype
            encoded_output = encoded_output.real
            logger.debug(
                f"Removing complex part from output_immersion: {old_dtype} to {encoded_output.dtype}"
            )
        if encoded_output.dtype != input.dtype:
            logger.debug(
                f"Casting output_immersion dtype from {encoded_output.dtype} to {input.dtype}"
            )
            encoded_output = encoded_output.to(dtype=input.dtype)

        return self._decoder(encoded_output).swapaxes(-1, -2)

    def _dmd_training_snapshots(self, snapshots):
        if self._n_prediction_snapshots > 0:
            return snapshots[..., :, : -self._n_prediction_snapshots]
        return snapshots

    def _prediction_snapshots(self, snapshots):
        if self._n_prediction_snapshots > 0:
            return snapshots[..., :, -self._n_prediction_snapshots :]
        return torch.zeros(0)

    def _compute_loss(self, output, input):
        logger.debug(f"Input shape: {input.shape}")
        logger.debug(f"Output shape: {output.shape}")

        decoder_loss = mse_loss(self._decoder(self._encoder(input.swapaxes(-1,-2))).swapaxes(-1,-2), input)

        batched_psp = self._dmd.operator.phase_space_prediction
        psp_loss = torch.linalg.matrix_norm(batched_psp).sum()

        reconstruction_loss = mse_loss(
            self._dmd_training_snapshots(output),
            self._dmd_training_snapshots(input),
        )

        prediction_loss = mse_loss(
            self._prediction_snapshots(output),
            self._prediction_snapshots(input),
        )

        return (
            self._encoding_weight * decoder_loss
            + self._phase_space_weight * psp_loss
            + self._reconstruction_weight * reconstruction_loss
            + self._prediction_weight * prediction_loss
        )

    @timer
    def train_step(self, loader):
        self.train()
        loss_sum = 0.0
        for i, minibatch in enumerate(loader):
            self._optimizer.zero_grad()
            output = self(self._dmd_training_snapshots(minibatch))
            loss = self._compute_loss(output, minibatch)
            loss.backward()
            self._optimizer.step()
            loss_sum += loss.item()
        return loss_sum / (i + 1)

    @timer
    def eval_step(self, loader):
        self.eval()
        loss_sum = 0.0
        prediction_sum = 0.0
        for i, minibatch in enumerate(loader):
            output = self(self._dmd_training_snapshots(minibatch))
            loss = self._compute_loss(output, minibatch)
            loss_sum += loss.item()

            prediction_sum += mse_loss(
                self._prediction_snapshots(output),
                self._prediction_snapshots(minibatch),
            ).item()

        loss_avg = loss_sum / (i + 1)
        return loss_avg, prediction_sum / (i + 1)

    def save_model(self, label="dldmd"):
        temp = self._dmd
        self._dmd = self._eval_dmd
        torch.save(self, label + ".pl")
        self._dmd = temp

    def load_model_for_eval(self, label="dldmd"):
        map_location = (
            "cpu"
            if self._eval_on_cpu or not torch.cuda.is_available()
            else "cuda"
        )
        return torch.load(label + ".pl", map_location=map_location)

The training loop receives a blank `dldmd` instance and two 3D tensors `training_data` and `test_data` (the first dimension is dedicated to batching). Other parameters can be used as well to tune the training (number of `epochs`, `batch_size`, etc).

In [19]:
def train_dldmd(dldmd, training_data, test_data,
                batch_size=256, epochs=1000, acceptable_loss=0, print_prediction_loss=True, 
                print_every=True, label="dldmd", eval_on_cpu=True):
    
    train_dataloader = DataLoader(
        training_data, batch_size=batch_size, shuffle=True
    )
    test_dataloader = DataLoader(
        test_data, batch_size=batch_size, shuffle=True
    )

    train_loss_arr = []
    eval_loss_arr = []
    
    if torch.cuda.is_available():
        dldmd.to("cuda", dtype=training_data.dtype)
    else:
        dldmd.to(dtype=training_data.dtype)

    for epoch in range(1, epochs + 1):
        training_time, train_loss = dldmd.train_step(train_dataloader)
        train_loss_arr.append(train_loss)

        model_label = f"{label}_e{epoch}"
        dldmd.save_model(model_label)
        eval_model = dldmd.load_model_for_eval(model_label)

        eval = eval_model.eval_step(test_dataloader)
        eval_time, (eval_loss, prediction_loss) = eval

        if not eval_loss_arr or min(eval_loss_arr) > eval_loss:
            dldmd.save_model(f"{label}_best")
        eval_loss_arr.append(eval_loss)

        if epoch % print_every == 0:
            logger.info(
                f"[{epoch}] loss: {eval_loss:.7f}, train_time: {training_time:.2f} ms, eval_time: {eval_time:.2f} ms"
            )
            if print_prediction_loss:
                logger.info(
                    f"[{epoch}] prediction loss: {prediction_loss:.7f}"
                )
        else:
            os.remove(model_label + ".pl")

        if eval_loss < acceptable_loss:
            break

    np.save(f"{label}_train_loss.npy", train_loss_arr)
    np.save(f"{label}_eval_loss.npy", eval_loss_arr)
    
    return dldmd

We also define a very simple MLP. We're going to use an hardcoded 3-layers perceptron like [1], but there's no constraint on the number of layers. The parameters required by the MLP are the input and output dimensions.

In [6]:
class MLP(Module):
    def __init__(self, input_size, output_size, hidden_layer_size):
        super().__init__()
        self.layers = Sequential(
            Linear(input_size, hidden_layer_size),
            ReLU(),
            Linear(hidden_layer_size, hidden_layer_size),
            ReLU(),
            Linear(hidden_layer_size, output_size),
        )

    def forward(self, x):
        return self.layers(x)

## Data
First of all we import a special module which we're going to use to generate data to feed DMD.

In [40]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parent) + "/src")
from data import data_maker_duffing_driven

In [42]:
training_count = 100
eval_count = 50

data = data_maker_duffing_driven(
    x_lower1=-1,
    x_upper1=1,
    x_lower=-1,
    x_upper2=1,
    n_ic=training_count + eval_count,
    dt=0.05,
    tf=200,
)
data = torch.from_numpy(data).swapaxes(-1, -2)
training_data = data[: training_count]
test_data = data[training_count :]

if torch.cuda.is_available():
    device = torch.device("cuda")
    training_data = training_data.to(device)
    if not args.eval_on_cpu:
        test_data = test_data.to(device)

In [43]:
hidden = 128
immersion_size = 3

encoder = MLP(data.shape[-2], immersion_size, hidden)
decoder = MLP(immersion_size, data.shape[-2], hidden)

In [44]:
dmd = DMD(svd_rank=-1)
dldmd = DLDMD(
    encoder=encoder,
    dmd=dmd,
    decoder=decoder,
    encoding_weight=1,
    reconstruction_weight=1.e-1,
    prediction_weight=1,
    phase_space_weight=1.e-3,
    n_prediction_snapshots=5
)

[1230415297.py:22 -             __init__() ] Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 1e-09
)
[1230415297.py:31 -             __init__() ] DMD instance: <class 'pydmd.dmd.DMD'>
[1230415297.py:37 -             __init__() ] DMD will predict 5 snapshots during training.
[1230415297.py:41 -             __init__() ] ----- DLDMD children -----
[1230415297.py:42 -             __init__() ] (MLP(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=3, bias=True)
  )
), MLP(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): 

In [ ]:
train_dldmd(dldmd, training_data, test_data, epochs=100)

[snapshots.py:40 -             __init__() ] Snapshots: torch.Size([100, 3, 3996]), snapshot shape: torch.Size([3, 3996])


In [36]:
d2 = data_maker_fluid_flow_full(
    x1_lower=-1.1,
    x1_upper=1.1,
    x2_lower=-1.1,
    x2_upper=1.1,
    x3_lower=0.0,
    x3_upper=2,
    n_ic=100,
    dt=0.01,
    tf=6,
)
d2 = torch.from_numpy(d2).swapaxes(-1,-2)
d2.shape

torch.Size([100, 3, 601])

In [37]:
dmd2 = DMD(svd_rank=-1).fit(d2[..., :-5], batch=True)
dmd2.dmd_time['tend'] += 5
rec = dmd2.reconstructed_data[..., -5:].real
print(mse_loss(rec,d2[..., -5:]))

dldmd_rec = dldmd(d2[..., :-5])[..., -5:]
print(mse_loss(dldmd_rec,d2[..., -5:]))

[snapshots.py:40 -             __init__() ] Snapshots: torch.Size([100, 3, 596]), snapshot shape: torch.Size([3, 596])
[snapshots.py:40 -             __init__() ] Snapshots: torch.Size([100, 3, 596]), snapshot shape: torch.Size([3, 596])


tensor(0.0534, dtype=torch.float64)
tensor(0.0010, dtype=torch.float64, grad_fn=<MseLossBackward0>)


# References
[1] Alford-Lago, Daniel J., et al. "Deep learning enhanced dynamic mode decomposition." Chaos: An Interdisciplinary Journal of Nonlinear Science 32.3 (2022): 033116.